# Introduction

This notebook contains code to combine open data from [`sensor.community`](https://sensor.community/) air quality sensors with wind data from the [Dutch Royal Meteorological Society (KNMI)](https://knmi.nl/). By default it will use the KNMI public API key (the one in the notebook is valid until June 2022, you can [find the latest public API key here](https://developer.dataplatform.knmi.nl/get-started#obtain-an-api-key)). You can also obtain a personal API key very easily, which will not expire (follow the instructions included below the public API key).

# Requirements

The cell below imports all required modules. All of these should be included in a basic Anaconda Python 3 installation, except for the `mpu` module, which you will need to install from the command line using `pip`.

In [26]:
import os
import sys
import requests
import datetime
import pandas
import gzip
import dateutil.relativedelta
import matplotlib.pyplot as plt
import numpy
import mpu

# Configuration

In [72]:
# KNMI API key (replace by your personal key if you have one, this public access key is valid until June 2022)
knmi_api_key = "eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6ImNjOWE2YjM3ZjVhODQwMDZiMWIzZGIzZDRjYzVjODFiIiwiaCI6Im11cm11cjEyOCJ9"

# Base URL for KNMI open wind data
knmi_wind_base_url = "https://api.dataplatform.knmi.nl/open-data/v1/datasets/windgegevens/versions/1.0/files"

# Directory to cache data downloaded from KNMI; directory will be created automatically if it doesn't exist
knmi_cache_dir = "./knmi_cache"

# Sensor data URL template; parameters: day,day,sensor ID
sensor_url_template = "https://archive.sensor.community/{}/{}_sds011_sensor_{}.csv"

# Directory to write plots to; directory will be created automatically if it doesn't exist
plots_dir = "./plots"

# KNMI data fetch

The function in the next cell downloads the KNMI open data using the configured API key and loads it into a Pandas dataframe.

In [13]:
def get_knmi_wind_data(year, month):
    # Cache filename
    cache_file = '{}/kis_tow_{:04d}{:02d}.gz'.format(knmi_cache_dir, year, month)
    
    if not os.path.exists(cache_file):
        if not os.path.exists(knmi_cache_dir):
            os.makedirs(knmi_cache_dir)
        
        # Compose URL to fetch a temporary download link from the KNMI API server
        month_url = '{}/kis_tow_{:04d}{:02d}.gz/url'.format(knmi_wind_base_url, year, month)
    
        print('Fetching temporary download URL from {} ...'.format(month_url))
    
        get_file_url_res = requests.get(month_url, headers={"Authorization": knmi_api_key})
    
        if get_file_url_res.status_code != 200:
            print("Failed ({})".format(get_file_url_res.status_code))
            return None
    
        tmp_url = get_file_url_res.json().get("temporaryDownloadUrl", None)

        if tmp_url is None:
            print('Response from server is missing the attribute "temporaryDownloadUrl"')
            return None
        
        print('Fetching data from {} ...'.format(tmp_url))
        
        get_file_res = requests.get(tmp_url)
        
        if get_file_res.status_code != 200:
            print("Fetch failed ({})".format(get_file_res.status_code))
            return None
        
        # Write file to cache
        with open(cache_file, 'wb') as fd:
            fd.write(get_file_res.content)
            
        print('Fetched {} ({:,d} bytes)'.format(cache_file, os.stat(cache_file).st_size))
    else:
        print('Wind data for {:04d}-{:02d} is cached'.format(year, month))
        
    # Parse wind data into a Pandas dataframe
    widths = []
    widths.append(len('DTG                '))
    widths.append(len('LOCATION            '))
    widths.append(len('NAME                                            '))
    widths.append(len('LATITUDE            '))
    widths.append(len('LONGITUDE           '))
    widths.append(len('ALTITUDE            '))
    widths.append(len('FF_SENSOR_10        '))
    widths.append(len('FF_10M_10           '))
    widths.append(len('DD_10               '))
    widths.append(len('DDN_10              '))
    widths.append(len('DD_STD_10           '))
    widths.append(len('DDX_10              '))
    widths.append(len('FF_10M_STD_10       '))
    widths.append(len('FX_10M_10           '))
    widths.append(len('FX_10M_MD_10        '))
    widths.append(len('FX_SENSOR_10        '))
    widths.append(len('FX_SENSOR_MD_10     '))
    widths.append(len('SQUALL_10           '))
    
    print('Parsing wind data...')
    
    df = pandas.read_fwf(cache_file, comment='#', compression='gzip', widths=widths,\
                         parse_dates=[0], infer_datetime_format=True, names=\
                        ['DTG','LOCATION','NAME','LATITUDE','LONGITUDE',\
                         'ALTITUDE','FF_SENSOR_10','FF_10M_10','DD_10',\
                         'DDN_10','DD_STD_10','DDX_10','FF_10M_STD_10',\
                         'FX_10M_10','FX_10M_MD_10','FX_SENSOR_10',\
                         'FX_SENSOR_MD_10','SQUALL_10'])
    
    print('Loaded DataFrame with {} rows and {} columns'.format(df.shape[0], df.shape[1]))
    
    return df

# Sensor data fetch

The function in the cell below fetches data for the sensor with the specified identifier for the specified month and returns it as a Pandas dataframe

In [28]:
def fetch_sensor_data(year, month, sensor_id):
    day = datetime.date(year, month, 1)
    end_date = day + dateutil.relativedelta.relativedelta(months=+1) - datetime.timedelta(days=1)
    
    day_frames = []
    
    print('Fetching sensor data for {:04d}-{:02d} for ID {} from the sensor.community archive'.format(year, month, sensor_id))
    
    while day <= end_date:
        day_url = sensor_url_template.format(day, day, sensor_id)
        
        print('Fetching data frame from {}'.format(day_url))
        
        try:
            df = pandas.read_csv(day_url, delimiter=';', parse_dates=[5])
            day_frames.append(df)
        except:
            print('Failed, perhaps there is no data for ID {} on {}'.format(sensor_id, day))
        
        day += datetime.timedelta(days=1)
        
    df = pandas.concat(day_frames)
    
    print('Loaded DataFrame with {} rows and {} columns'.format(df.shape[0], df.shape[1]))
    
    return df

# Get wind data for the closest station

The function in the cell below takes as input sensor data and fetches wind data for the closest station

In [37]:
def get_closest_wind_data(year, month, sensor_df):
    # Get wind data for the specified month
    wind_df = get_knmi_wind_data(year, month)
    
    if wind_df is None:
        print('No wind data for {:04d}-{:02d}'.format(year, month))
        return None
    
    # Drop stations that do not have wind direction information
    wind_df = wind_df[wind_df['DD_10'].notna()]
    
    # Determine the longitude and latitude of the sensor
    sensor_pos_df = sensor_df[['lat', 'lon']].drop_duplicates()
    sensor_lat = sensor_pos_df['lat'].values[0]
    sensor_lon = sensor_pos_df['lon'].values[0]
    
    print('Sensor is at {},{}'.format(sensor_lat, sensor_lon))
    
    # Determine wind station locations
    wind_lat_lon = wind_df[['LATITUDE','LONGITUDE','LOCATION','NAME']].drop_duplicates()
    
    dist = 0
    station_loc = None
    station_name = None
    
    for index,row in wind_lat_lon.iterrows():
        wind_lat = row["LATITUDE"]
        wind_lon = row["LONGITUDE"]
        
        station_to_sensor_dist = mpu.haversine_distance((sensor_lat, sensor_lon), (wind_lat, wind_lon))
        
        if dist == 0 or station_to_sensor_dist < dist:
            station_loc = row["LOCATION"]
            station_name = row["NAME"]
            dist = station_to_sensor_dist
            
    print('Closest station to sensor is "{}" at distance {:0.1f} km'.format(station_name, dist))
    
    station_df = wind_df[wind_df.LOCATION == station_loc]
    
    print('Filtered wind DataFrame with {} rows and {} columns'.format(station_df.shape[0], station_df.shape[1]))
    
    return station_df

# Get combined data for a month

The function in the cell below takes a year, month and sensor ID as input and outputs a DataFrame that contains combined wind and sensor data that can be used for plotting

In [51]:
def get_month_data(year, month, sensor_id):
    # Get sensor data
    sensor_df = fetch_sensor_data(year, month, sensor_id)
    
    # Get wind data
    wind_df = get_closest_wind_data(year, month, sensor_df)
    
    # Start with an empty DataFrame
    result_df = pandas.DataFrame(columns=['sensorId', 'hourOfDay','windDirection','windDirectionRadians','pm25_mean','pm25_median', 'pm25_max', 'pm10_mean', 'pm10_median', 'pm10_max'])
    
    # Iterate over the wind data and match this with sensor data
    cur_day = None
    
    sys.stdout.write('Processing {:04d}-{:02d}: '.format(year, month))
    sys.stdout.flush()
    
    for index,row in wind_df.iterrows():
        # Determine wind time window for this row
        start_ts = row['DTG'] - datetime.timedelta(minutes=10)
        end_ts = row['DTG']
        
        # Filter sensor data for this window
        sensor_window = sensor_df[(sensor_df['timestamp'] >= start_ts) & (sensor_df['timestamp'] < end_ts)]
        
        avg_pm25 = sensor_window['P2'].mean()
        med_pm25 = sensor_window['P2'].median()
        max_pm25 = sensor_window['P2'].max()
        
        avg_pm10 = sensor_window['P1'].mean()
        med_pm10 = sensor_window['P1'].median()
        max_pm10 = sensor_window['P1'].max()
        
        if avg_pm25 > 0 or avg_pm10 > 0:
            newRowDict = { 'sensorId': sensor_id, 'yearMonth': '{:04d}-{:02d}'.format(year, month), 'hourOfDay': start_ts.hour, \
                          'windDirection': row['DD_10'], 'windDirectionRadians': numpy.deg2rad(row['DD_10']), \
                          'pm25_mean': avg_pm25, 'pm25_median': med_pm25, 'pm25_max': max_pm25, \
                          'pm10_mean': avg_pm10, 'pm10_median': med_pm10, 'pm10_max': max_pm10 }
            result_df = result_df.append(newRowDict, ignore_index=True)

        row_day = start_ts.day
        
        if cur_day != row_day:
            sys.stdout.write('{} '.format(row_day))
            sys.stdout.flush()
            cur_day = row_day
            
    print('done')
    
    print('Collected result DataFrame with {} rows and {} columns'.format(result_df.shape[0], result_df.shape[1]))
    
    return result_df

# Plotting functions

The cell below defines the plotting functions. The plotting function `pm_scatter` plots a scatter plot with the wind direction in degrees on the y-axis and the PM sensor value in microgram/cubic metre on the x-axis. The plotting function `pm_scatter_polar` plots a polar plot with the wind direction as angle and the PM sensor value in microgram/cubic metre as value.

In [54]:
def pm_plot_indiv(df, x, y, c, cmap, ax, title):
    df.plot.scatter(x=x, y=y, c=c, colormap=cmap, ax=ax)
    ax.set_title(title)
    ax.set(xlabel='',ylabel='')
    
def pm_scatter(df):
    month = df['yearMonth'].values[0]
    sensor_id = df['sensorId'].values[0]
    
    fig, axes = plt.subplots(2, 3, figsize=(20,7.5))
    fig.suptitle('Particulate matter distribution versus wind direction ({} for sensor {})'.format(month, sensor_id))
    
    pm_plot_indiv(df, 'pm25_mean',   'windDirection', 'hourOfDay', 'plasma', axes[0,0], 'PM2.5 mean')
    pm_plot_indiv(df, 'pm25_median', 'windDirection', 'hourOfDay', 'plasma', axes[0,1], 'PM2.5 median')
    pm_plot_indiv(df, 'pm25_max',    'windDirection', 'hourOfDay', 'plasma', axes[0,2], 'PM2.5 max')
    
    pm_plot_indiv(df, 'pm10_mean',   'windDirection', 'hourOfDay', 'plasma', axes[1,0], 'PM10 mean')
    pm_plot_indiv(df, 'pm10_median', 'windDirection', 'hourOfDay', 'plasma', axes[1,1], 'PM10 median')
    pm_plot_indiv(df, 'pm10_max',    'windDirection', 'hourOfDay', 'plasma', axes[1,2], 'PM10 max')
    
    if not os.path.exists(plots_dir):
        os.makedirs(plots_dir)

    fig.tight_layout()
    fig.savefig('{}/pm-vs-wind-{}-{}.pdf'.format(plots_dir, sensor_id, month))
    plt.close(fig)
    
def pm_scatter_polar(df):
    month = df['yearMonth'].values[0]
    sensor_id = df['sensorId'].values[0]
    
    fig, axes = plt.subplots(2, 3, figsize=(20,8), subplot_kw=dict(polar=True))
    fig.suptitle('Particulate matter distribution versus wind direction ({} for sensor {})'.format(month, sensor_id))
    
    pm_plot_indiv(df, 'windDirectionRadians', 'pm25_mean',   'hourOfDay', 'plasma', axes[0,0], 'PM2.5 mean')
    pm_plot_indiv(df, 'windDirectionRadians', 'pm25_median', 'hourOfDay', 'plasma', axes[0,1], 'PM2.5 median')
    pm_plot_indiv(df, 'windDirectionRadians', 'pm25_max',    'hourOfDay', 'plasma', axes[0,2], 'PM2.5 max')
    
    pm_plot_indiv(df, 'windDirectionRadians', 'pm10_mean',   'hourOfDay', 'plasma', axes[1,0], 'PM10 mean')
    pm_plot_indiv(df, 'windDirectionRadians', 'pm10_median', 'hourOfDay', 'plasma', axes[1,1], 'PM10 median')
    pm_plot_indiv(df, 'windDirectionRadians', 'pm10_max',    'hourOfDay', 'plasma', axes[1,2], 'PM10 max')
    
    for ax in axes.flat:
        ax.set_theta_direction(-1)
        ax.set_theta_zero_location('N')
        ax.grid(linewidth=1)
        ax.set_xticks(ax.get_xticks().tolist())
        ax.set_xticklabels(['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'])
        
    if not os.path.exists(plots_dir):
        os.makedirs(plots_dir)
            
    fig.tight_layout()
    fig.savefig('{}/pm-vs-wind-polar-{}-{}.pdf'.format(plots_dir, sensor_id, month))
    plt.close(fig)

# Putting it all together

The code snippet below shows an example of how to put it all together. It creates plots for my sensor (ID 39024) for July to December 2020.

In [ ]:
for month in [7, 8, 9, 10, 11, 12]:
    df = get_month_data(year = 2020, month = month, sensor_id = 39024)
    
    pm_scatter(df)
    pm_scatter_polar(df)

Fetching sensor data for 2020-07 for ID 39024 from the sensor.community archive
Fetching data frame from https://archive.sensor.community/2020-07-01/2020-07-01_sds011_sensor_39024.csv
Fetching data frame from https://archive.sensor.community/2020-07-02/2020-07-02_sds011_sensor_39024.csv
Fetching data frame from https://archive.sensor.community/2020-07-03/2020-07-03_sds011_sensor_39024.csv
Fetching data frame from https://archive.sensor.community/2020-07-04/2020-07-04_sds011_sensor_39024.csv
Fetching data frame from https://archive.sensor.community/2020-07-05/2020-07-05_sds011_sensor_39024.csv
Fetching data frame from https://archive.sensor.community/2020-07-06/2020-07-06_sds011_sensor_39024.csv
Fetching data frame from https://archive.sensor.community/2020-07-07/2020-07-07_sds011_sensor_39024.csv
Fetching data frame from https://archive.sensor.community/2020-07-08/2020-07-08_sds011_sensor_39024.csv
Fetching data frame from https://archive.sensor.community/2020-07-09/2020-07-09_sds011_s